In [1]:
!pip install --q catboost
!pip install --q cleanlab

In [2]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import cleanlab
import re

from catboost import CatBoostClassifier
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_val_predict, KFold
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, classification_report

In [3]:
pd.options.mode.chained_assignment = None

# Constants

In [4]:
wb_mapping = {
    0: 'Unknown/not stated',
    1: 'Clay',
    2: 'Mortar + Cement',
    3: 'Mortar + Cement, Clay',
    5: 'Mud + Mortar, Clay',
    7: 'Mud + Mortar, Clay, Cement + Mortar'
}

wm_mapping = {
    0: 'Unknown/not stated',
    1: 'Red Bricks',
    2: 'Stone Bricks',
    3: 'Red Bricks, Stone Bricks'
}

# Pre-defined Function (Basic pre-processing)

In [5]:
#@title extract_num_floors
def extract_num_floors(floor):
    """
    This function extracts the number of floors from the 'floors_before_eq' column
    and converts it to an integer. If the value is missing or cannot be converted
    to an integer, it returns None.

    Arguments:
    - floor: The value in the 'floors_before_eq' column for a particular building

    Returns:
    - An integer representing the number of floors for the given building, or None
      if the value is missing or cannot be converted to an integer.
    """
    import re

    if floor is None:
        return None
    else:
        floor = str(floor)
        floor = floor.lower()
        if 'one' in floor or 'first' in floor or 'ground' in floor:
            return 1
        elif 'two' in floor or 'second' in floor:
            return 2
        elif 'three' in floor or 'third' in floor:
            return 3
        elif 'four' in floor or 'fourth' in floor:
            return 4
        elif 'five' in floor or 'fifth' in floor:
            return 5
        else:
            floor = re.findall(r'\d+', floor)
            if len(floor) > 0:
                return int(floor[0])
            else:
                return None

In [6]:
#@title standardize_value
def standardize_value(data):
    """
    This function takes a pandas dataframe as input and returns a new pandas dataframe
    where all the categorical values are standardized. For example, if 'RCC' and 'RC' 
    are used interchangeably in the 'foundation_type' column, this function will standardize 
    all the 'RCC' values to 'RC' to avoid any inconsistencies.
    """

    # type_of_foundation
    data['type_of_foundation'] = data['type_of_foundation'].replace('RC', 'Reinforced Concrete')
    data['type_of_foundation'] = data['type_of_foundation'].replace(['Bamboo/TImber', 'Bamboo or Timber', 'Bamboo/Timber'], 'Bamboo/Timber')
    data['type_of_foundation'] = data['type_of_foundation'].replace(['Other', 'Others'], 'Other')
    data['type_of_foundation'] = data['type_of_foundation'].replace(['Cement-Stone/Brick', 'Cement-Stone or Cement-Brick'], 'Cement-Stone/Brick')
    
    # type_of_roof
    data['type_of_roof'] = data['type_of_roof'].replace(['Bamboo/Timber Light roof', 'Bamboo/TImber-Light Roof', 'Bamboo or Timber Light roof'], 'Bamboo/Timber Light roof')
    data['type_of_roof'] = data['type_of_roof'].replace(['Bamboo/Timber Heavy roof ', 'Bamboo/TImber-Heavy Roof', 'Bamboo or Timber Heavy roof'], 'Bamboo/Timber Heavy roof')
    data['type_of_roof'] = data['type_of_roof'].replace(['reinforced cement concrete/rb/rbc', 'rcc/rb/rbc', 'Reinforced Brick Slab/rcc/rbc', 'Reinforced brick concrete/rcc/rbc'], 'rcc/rb/rbc')
    
    # type_of_ground_floor
    data['type_of_ground_floor'] = data['type_of_ground_floor'].replace(['mud', 'Mud'], 'Mud')
    data['type_of_ground_floor'] = data['type_of_ground_floor'].replace(['reinforced concrete', 'Reinforced Concrete', 'RC'], 'Reinforced Concrete ')
    data['type_of_ground_floor'] = data['type_of_ground_floor'].replace(['Brick/Stone', 'brick/stone', 'Brick or Stone'], 'Brick/Stone')
    data['type_of_ground_floor'] = data['type_of_ground_floor'].replace(['TImber', 'Timber', 'Wood', 'Lumber'], 'Wood')

    # type_of_other_floor
    data['type_of_other_floor'] = data['type_of_other_floor'].replace(['TImber/Bamboo-Mud', 'Timber Mud or Bamboo-Mud', 'Wood or Bamboo Mud'], 'Wood or Bamboo Mud')
    data['type_of_other_floor'] = data['type_of_other_floor'].replace(['Timber-Planck', 'Lumber-plank', 'Wood-Plank'], 'Wood-Plank')
    data['type_of_other_floor'] = data['type_of_other_floor'].replace(['rcc/rb/rbc', 'reinforced cement concrete/rb/rbc', 'Reinforced brick concrete/rcc/rbc'], 'rcc/rb/rbc')
    data['type_of_other_floor'] = data['type_of_other_floor'].replace(['Wood or Bamboo Mud', 'Wood-Mud or Bamboo Mud'], 'Wood-Mud or Bamboo Mud ')

    # legal_ownership_status
    data['legal_ownership_status'] = data['legal_ownership_status'].replace(['Private', 'Private Use', 'Prvt', 'Privste'], 'Private')
    data['legal_ownership_status'] = data['legal_ownership_status'].replace(['Public', 'Public Use', 'Public Space'], 'Public')
    data['legal_ownership_status'] = data['legal_ownership_status'].replace(['Institutional', 'Institutional Use', 'Institutionals'], 'Institutional')
    data['legal_ownership_status'] = data['legal_ownership_status'].replace(['Other', 'Unknown', 'Unspecified'], 'Other')

    return data

In [7]:
#@title find_label_issue_train
def find_label_issue_train(data):
    """
    Find label errors in the training data using the Confident Learning framework.

    Arguments:
    - data: A pandas DataFrame containing the training data.

    Returns:
    - A list of indices of examples in the training data that are likely to have label errors.

    References: 
    """
    # Step 1: Apply a machine learning model to the training data to obtain predicted labels.
    # This model should be trained using standard techniques and should not take into account the issue of label errors.
    # Here, we assume that the model is already trained and is available for use.

    # Step 2: Compute the empirical distribution of the predicted labels.
    # This distribution will be used later to estimate the noise distribution.

    # Step 3: Compute the joint distribution of the predicted labels and the true labels.
    # This distribution can be estimated using the empirical distribution of the predicted labels and an estimate of the noise distribution.

    # Step 4: Rank the examples in the training data by their likelihood of being mislabeled.
    # This can be done by computing the conditional probability of the true label given the predicted label.

    # Step 5: Identify examples that are likely to be mislabeled.
    # This can be done by selecting examples with low confidence and high probability of being mislabeled.

    # Step 6: Return the indices of the examples that are likely to be mislabeled.
    # These examples can be further examined to determine if they are indeed mislabeled, and corrected if necessary.
    # Alternatively, they can be removed from the training data to improve the performance of the machine learning model.

    X = data[data.columns.difference(['damage_grade'])]
    y = data['damage_grade']

    numerical_cols = X.select_dtypes(include=['int64', 'float64']).columns.tolist()
    categorical_cols = X.select_dtypes(include=['object']).columns.tolist()

    numerical_transformer = StandardScaler()
    categorical_transformer = OneHotEncoder(sparse=False)

    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numerical_transformer, numerical_cols),
            ('cat', categorical_transformer, categorical_cols)
        ])

    X = preprocessor.fit_transform(X)

    label_encoder = LabelEncoder()
    y = label_encoder.fit_transform(y)

    model = XGBClassifier(tree_method="hist", enable_categorical=True)
    pred_probs = cross_val_predict(model, X, y, method='predict_proba')

    cl_issue_idx = cleanlab.filter.find_label_issues(y, pred_probs, return_indices_ranked_by='self_confidence')
    
    selected_rows = data.iloc[cl_issue_idx]
    inverted_selection = data[~data.index.isin(cl_issue_idx)]

    return inverted_selection

In [8]:
#@title complete_pipeline
def complete_pipeline(data):
    """
    This function takes a pandas dataframe as input, processes it through
    various steps including missing value imputation, feature engineering,
    and standardization, and returns the processed dataframe ready for
    machine learning modeling. The function is designed to handle both
    training and testing data by distinguishing between them based on the
    presence of the 'damage_grade' column.
    """

    # floors_before_eq (total)
    data['floors_before_eq (total)'] = data['floors_before_eq (total)'].apply(lambda x: extract_num_floors(x))

    # missing value
    data = data[data['floors_before_eq (total)'].notna()]
    data = data[data['plinth_area (ft^2)'].notna()]
    data['technical_solution_proposed'] = data['technical_solution_proposed'].fillna('Unspecified')

    # replace and lambda
    data['plinth_area (ft^2)'] = data['plinth_area (ft^2)'].str.extract('(\d+)').astype(float)
    
    data['no_family_residing'] = data['no_family_residing'].replace('None', 0).astype(float)
    data['wall_binding'] = data['wall_binding'].replace(wb_mapping)
    data['wall_material'] = data['wall_material'].replace(wm_mapping)

    # data['type_of_reinforcement_concrete'] = data['type_of_reinforcement_concrete'].apply(lambda x: f'Type: {str(x)}')
    
    # value standardization
    data = standardize_value(data)

    # same group of features, different label
    data = data.groupby(list(data.columns.difference(['damage_grade']))).filter(lambda x: len(set(x['damage_grade'])) == 1)

    # label issue in train data
    data = data.reset_index()
    data.drop(['index'], axis=1, inplace=True)
    data = find_label_issue_train(data)
    
    return data

In [9]:
#@title complete_pipeline_test
def complete_pipeline_test(data):
    """
    This function takes a pandas dataframe as input, processes it through
    various steps including missing value imputation, feature engineering,
    and standardization, and returns the processed dataframe ready for
    machine learning modeling. The function is designed to handle both
    training and testing data by distinguishing between them based on the
    presence of the 'damage_grade' column.
    """
    
    data['floors_before_eq (total)'] = data['floors_before_eq (total)'].apply(lambda x: extract_num_floors(x))

    # others
    data['plinth_area (ft^2)'] = data['plinth_area (ft^2)'].str.extract('(\d+)').astype(float)
    
    data['no_family_residing'] = data['no_family_residing'].replace('None', 0).astype(float)
    data['wall_binding'] = data['wall_binding'].replace(wb_mapping)
    data['wall_material'] = data['wall_material'].replace(wm_mapping)
    
    data['type_of_reinforcement_concrete'] = data['type_of_reinforcement_concrete'].apply(lambda x: f'Type: {str(float(x))}')

    # value standardization
    data = standardize_value(data)
    
    return data

In [10]:
#@title cross_val_multiclass
def cross_val_multiclass(model, X, y, num_folds=5):
    from sklearn.model_selection import KFold, cross_val_score
    from sklearn.metrics import f1_score
    import numpy as np

    kf = KFold(n_splits=num_folds, shuffle=True, random_state=42)
    f1_scores = []
    for train_idx, test_idx in kf.split(X):
        X_train, X_test = X[train_idx], X[test_idx]
        y_train, y_test = y[train_idx], y[test_idx]
        model.fit(X_train, y_train, verbose=0)
        y_pred = model.predict(X_test)
        f1_scores.append(f1_score(y_test, y_pred, average='macro'))
    return np.mean(f1_scores), np.std(f1_scores)

In [11]:
#@title categorical_correlation
def categorical_correlation(var1, var2):
    import scipy.stats as stats

    contingency_table = pd.crosstab(var1, var2)
  
    chi2, p, dof, expected = stats.chi2_contingency(contingency_table)
    
    n = contingency_table.sum().sum()
    phi2 = chi2/n
    r, k = contingency_table.shape
    phi2corr = max(0, phi2 - ((k-1)*(r-1))/(n-1))
    rcorr = r - ((r-1)**2)/(n-1)
    kcorr = k - ((k-1)**2)/(n-1)
    V = np.sqrt(phi2corr / min((kcorr-1), (rcorr-1)))
    
    return {"Chi-squared test": {"test statistic": chi2, "p-value": p}, "Cramer's V": V}

# Training Flow

In [12]:
train = pd.read_csv('/kaggle/input/jointsugm/train.csv')
train.drop(['Unnamed: 0'], axis=1, inplace=True)

train = complete_pipeline(train)
train = train.reset_index()
train.drop(['index', 'technical_solution_proposed'], axis=1, inplace=True)

/opt/conda/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3553: DtypeWarning: Columns (1,3,5,6,7,8,9,10,11,12,13,16,17,19,20) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [13]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 193502 entries, 0 to 193501
Data columns (total 23 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   floors_before_eq (total)        193502 non-null  float64
 1   old_building                    193502 non-null  float64
 2   plinth_area (ft^2)              193502 non-null  float64
 3   height_before_eq (ft)           193502 non-null  float64
 4   land_surface_condition          193502 non-null  object 
 5   type_of_foundation              193502 non-null  object 
 6   type_of_roof                    193502 non-null  object 
 7   type_of_ground_floor            193502 non-null  object 
 8   type_of_other_floor             193502 non-null  object 
 9   position                        193502 non-null  object 
 10  building_plan_configuration     193502 non-null  object 
 11  legal_ownership_status          193502 non-null  object 
 12  has_secondary_us

In [14]:
train.drop(['building_plan_configuration', 'legal_ownership_status', 'type_of_reinforcement_concrete',
           'residential_type', 'public_place_type', 'industrial_use_type', 'govermental_use_type', 'flexible_superstructure'], axis=1, inplace=True)

In [15]:
train['total_building_area (ft^2)'] = train['plinth_area (ft^2)'] * train['height_before_eq (ft)']
train['area_to_height_ratio'] = train['plinth_area (ft^2)'] / train['height_before_eq (ft)']
train['height per floor (ft)'] = train['height_before_eq (ft)'] / train['floors_before_eq (total)']

In [16]:
train['gross_floor_area'] = train['plinth_area (ft^2)'] * train['floors_before_eq (total)']
train['floor_area_ratio'] = train['total_building_area (ft^2)'] / train['gross_floor_area']

In [17]:
train = train[train['height per floor (ft)'] <= 15]
train = train.reset_index()
train.drop(['index'], axis=1, inplace=True)

In [18]:
train = train.groupby(list(train.columns.difference(['damage_grade']))).filter(lambda x: len(set(x['damage_grade'])) == 1)
train.shape

(190900, 20)

In [19]:
train = train.reset_index()
train.drop(['index'], axis=1, inplace=True)

In [20]:
train = find_label_issue_train(train)
train = train.reset_index()
train.drop(['index'], axis=1, inplace=True)
train.shape

(152061, 20)

# Modeling

In [21]:
X = train[train.columns.difference(['damage_grade'])]
y = train['damage_grade']

numerical_cols = X.select_dtypes(include=['int64', 'float64']).columns.tolist()
categorical_cols = X.select_dtypes(include=['object']).columns.tolist()

numerical_transformer = StandardScaler()
categorical_transformer = OneHotEncoder(sparse=False)

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

X = preprocessor.fit_transform(X)

label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

In [22]:
# Create train-validation-test split with stratification
X_train, X_val, y_train, y_val = train_test_split(X, y, train_size=0.6, random_state=42, stratify=y)
X_val, X_test, y_val, y_test = train_test_split(X_val, y_val, test_size=0.5, random_state=42, stratify=y_val)

## Manual

### LightGBM

In [23]:
lgbm = LGBMClassifier()
lgbm.fit(X_train, y_train, verbose=0, eval_set=(X_val, y_val))

/opt/conda/lib/python3.7/site-packages/lightgbm/sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "


LGBMClassifier()

In [24]:
y_val_pred = lgbm.predict(X_val)
print(classification_report(y_val, y_val_pred, target_names=['1', '2', '3', '4', '5']))

              precision    recall  f1-score   support

           1       0.92      0.95      0.93      3800
           2       0.77      0.43      0.55      1590
           3       0.72      0.49      0.58      2739
           4       0.73      0.62      0.67      5046
           5       0.84      0.94      0.89     17237

    accuracy                           0.82     30412
   macro avg       0.80      0.69      0.72     30412
weighted avg       0.81      0.82      0.81     30412



### CatBoost

In [25]:
catboost = CatBoostClassifier()
catboost.fit(X_train, y_train, verbose=0, eval_set=(X_val, y_val))

In [26]:
y_val_pred = catboost.predict(X_val)
print(classification_report(y_val, y_val_pred, target_names=['1', '2', '3', '4', '5']))

              precision    recall  f1-score   support

           1       0.92      0.94      0.93      3800
           2       0.73      0.44      0.55      1590
           3       0.71      0.54      0.61      2739
           4       0.74      0.64      0.68      5046
           5       0.85      0.94      0.89     17237

    accuracy                           0.83     30412
   macro avg       0.79      0.70      0.73     30412
weighted avg       0.82      0.83      0.82     30412



### XGBoost

In [27]:
xgboost = XGBClassifier()
xgboost.fit(X_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', callbacks=None,
              colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
              early_stopping_rounds=None, enable_categorical=False,
              eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
              importance_type=None, interaction_constraints='',
              learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
              max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
              missing=nan, monotone_constraints='()', n_estimators=100,
              n_jobs=0, num_parallel_tree=1, objective='multi:softprob',
              predictor='auto', random_state=0, reg_alpha=0, ...)

In [28]:
y_val_pred = xgboost.predict(X_val)
print(classification_report(y_val, y_val_pred, target_names=['1', '2', '3', '4', '5']))

              precision    recall  f1-score   support

           1       0.92      0.95      0.94      3800
           2       0.76      0.45      0.57      1590
           3       0.72      0.53      0.61      2739
           4       0.74      0.63      0.68      5046
           5       0.84      0.94      0.89     17237

    accuracy                           0.83     30412
   macro avg       0.80      0.70      0.74     30412
weighted avg       0.82      0.83      0.82     30412



## CV

In [29]:
X_cv, _, y_cv, _ = train_test_split(X, y, train_size=0.8, random_state=42, stratify=y)

### LightGBM

In [30]:
f1_avg, f1_std = cross_val_multiclass(lgbm, X_cv, y_cv)

/opt/conda/lib/python3.7/site-packages/lightgbm/sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "
/opt/conda/lib/python3.7/site-packages/lightgbm/sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "
/opt/conda/lib/python3.7/site-packages/lightgbm/sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. 

In [31]:
print(f'Avg: {f1_avg}, Std: {f1_std}')

Avg: 0.7197526172233966, Std: 0.004027371172380708


### CatBoost

In [32]:
f1_avg, f1_std = cross_val_multiclass(catboost, X_cv, y_cv)

In [33]:
print(f'Avg: {f1_avg}, Std: {f1_std}')

Avg: 0.7296141598416661, Std: 0.002828993223292088


### XGBoost

In [34]:
f1_avg, f1_std = cross_val_multiclass(xgboost, X_cv, y_cv)

In [35]:
print(f'Avg: {f1_avg}, Std: {f1_std}')

Avg: 0.7343601420022494, Std: 0.0024467234035241547
